Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.



In [164]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn.utils import shuffle

In [165]:
with open('checkins.dat', 'r') as file1:
    with open('checkins-new.dat', 'w') as file2:
        for index, line in enumerate(file1):
            newLine = line.replace(' ','').replace('|', ',')
            if (index != 1):
                file2.write(newLine)
                
data = pd.read_csv('checkins-new.dat').dropna()

In [166]:
df = shuffle(data)
cols = [3, 4]

In [167]:
X = data.iloc[:100000,cols]

In [168]:
cluster = MeanShift(bandwidth=0.1)
cluster.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [169]:
centers = cluster.cluster_centers_

In [170]:
label_counts1 = np.unique(cluster.labels_, return_counts=True)
label_counts1 = zip(label_counts1[0], label_counts1[1])

In [171]:
sorted_label_count = sorted(label_counts1, key = lambda x:x[1], reverse=True)

In [172]:
array = [center for center, count in sorted_label_count if count > 15]

In [173]:
array1 = [(center, count) for center, count in sorted_label_count if count > 15]

In [174]:
array_centers = centers[array]

In [175]:
offices = [[33.751277,-118.188740],
[25.867736,-80.324116],
[51.503016,-0.075479],
[52.378894,4.885084],
[39.366487,117.036146],
[-33.868457,151.205134]]

In [176]:
listofnorm = []
for x in array_centers:
    m = np.linalg.norm(x - offices[0])
    for i in range(1, len(offices)):
        if np.linalg.norm(x - offices[i]) < m:
            m = np.linalg.norm(x - offices[i])
    listofnorm.append((x, m))

In [177]:
listofnorm_sort = sorted(listofnorm, key = lambda x:x[1])

In [178]:
listofnorm_sort[:20]

[(array([-33.86063043, 151.20477593]), 0.007834758163107856),
 (array([52.37296399,  4.89231722]), 0.009353316185992226),
 (array([ 25.84567226, -80.3188906 ]), 0.022674066158385495),
 (array([51.50299126, -0.12553729]), 0.05005829482278787),
 (array([  33.80987796, -118.14892381]), 0.07084773242719973),
 (array([ 25.78581242, -80.21793804]), 0.13410903336184654),
 (array([ 25.70534972, -80.28342874]), 0.16740596425035326),
 (array([ 26.01009825, -80.19999059]), 0.18887596060185083),
 (array([  33.88832534, -118.04892817]), 0.19577945647763628),
 (array([  33.87298601, -118.36209115]), 0.21181053682436798),
 (array([  33.97257482, -118.16837067]), 0.2222332907317907),
 (array([ 26.13884379, -80.33434684]), 0.2713007595066735),
 (array([  33.98393587, -118.00740497]), 0.2949788868004569),
 (array([ 26.12086266, -80.15890668]), 0.3022701186924605),
 (array([  33.81730643, -117.89124917]), 0.30473050307840693),
 (array([  34.06039755, -118.24870903]), 0.3148837903362732),
 (array([  33.67

In [179]:
def write_answer(answer,name):
    with open(name, "w") as fout:
        fout.write(" ".join([str(num) for num in answer]))

In [180]:
write_answer(listofnorm_sort[0][0],'banners.txt')